In [1]:
from Crypto.Util.number import *
import random
from tqdm import *

######################################################### part1 recover MT and get seed
RNG = random.Random()

def construct_a_row(RNG):
    row = []
    RNG.getrandbits(128*11)
    for i in range(bytes_to_long(b"SU")):
        RNG.getrandbits(128*10)
        row += [int(int(RNG.getrandbits(128)) & 1)]
    return row

L = []
for i in trange(19968):
    state = [0]*624
    temp = "0"*i + "1"*1 + "0"*(19968-1-i)
    for j in range(624):
        state[j] = int(temp[32*j:32*j+32],2)
    RNG.setstate((3,tuple(state+[624]),None))
    L.append(construct_a_row(RNG))

L = Matrix(GF(2),L)

100%|█████████████████████████████████████████████████████████████████████████████| 19968/19968 [05:06<00:00, 65.07it/s]


In [5]:
K = L.T
KK = K[:19937, [0]+list(range(32,19968))]

In [8]:
KK_inv = KK^(-1)

In [19]:
from pwn import *

context.log_level = "critical"

#sh = process(["sage", "chall.sage"])
sh = remote("1.95.46.185", 10005)
sh.recvuntil(b" :")
points = eval(sh.recvline().strip().decode())
known = [i[0] % 2 for i in points]

print(1 in known)
if(1 not in known):
    sh.close()

True


In [20]:
R = vector(GF(2), known[:19937])
res = (KK_inv * R).list()
res = [res[0]] + [0]*31 + res[1:]

init = "".join(list(map(str, res)))
state = []
for i in range(624):
    state.append(int(init[32*i:32*i+32],2))

from hashlib import md5
import random
RNG1 = random.Random()
RNG1.setstate((3,tuple(state+[624]),None))

g = [RNG1.getrandbits(128) for i in range(11)][::-1]

sh.recvuntil(b"Show me :)")
sh.sendline(str(md5(str(g).encode()).hexdigest()).encode())

print(sh.recvline().decode())
sh.close()


#SUCTF{bytes_to_long_of_SU_seems_bigger_than_19937_XD_try_random_matrix(GF(2),20000,20000)_and_see_its_rank!}

🚩 : b'SUCTF{bytes_to_long_of_SU_seems_bigger_than_19937_XD_try_random_matrix(GF(2),20000,20000)_and_see_its_rank!}'

